# Jakub Zbrzezny, nr indeksu: 286689

# Projekt

In [ ]:
# pip install keras

In [225]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pprint import pprint

from matplotlib import pyplot as plt

import statsmodels.api as sm
import statsmodels.formula.api as smf

# Import kerasa
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

from tensorflow.keras.datasets import mnist

import sklearn.metrics # Importowanie funkcji sklearn.metrics.roc_auc_score.
# Ta funkcja liczy nam AUC.

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, SimpleRNN, Dense

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

## Wczytanie danych

### Mały zbiór treningowy

In [172]:
# Dane .gz rozpakowałem.
D100k = pd.read_csv("D100k.tsv", sep = "\t")

In [173]:
D100k.head(20)

,Click,DisplayURL,AdId,AdvertiserId,Depth,Position,UserID,Gender,Age,AdKeyword_tokens,AdTitle_tokens,AdDescription_tokens,Query_tokens
0,0,4298118681424644510,7686695,385,3,3,490234,1,3,4133|95|17,4133|95|17|0|4732|95|146|4079,8|81|123|205|2|95|26|95|60|32|1|17|146|1|991|3...,4133
1,0,13677630321509009335,3517124,23778,3,1,490234,1,3,4133,145|65|3927|832|93,3683|4990|2793|11589|21|10741|26|16044|26|3168...,4133
2,0,11689327222955583742,21021375,27701,3,2,490234,1,3,4133,4133|95|1|339|125|21|83093,726|50|2218|1533|2275|4133|1299|509|95|2072|1|...,4133
3,0,4298118681424644510,7686695,385,1,1,16960371,2,2,4133|95|17,4133|95|17|0|4732|95|146|4079,8|81|123|205|2|95|26|95|60|32|1|17|146|1|991|3...,4133|4942
4,0,15132506310926074459,4424000,20940,1,1,3524325,1,3,121|4133|95,121|4133|95|8762|3957|4563|2233|192|28|138|3,62|1162|570|8|4133|95|1|81|102|1155|650|1255|1...,4133|4942
5,0,4298118681424644510,7686888,385,2,2,11198634,2,3,14530|882|167,14530|882|167|0|4732|95|146|4079,8|81|123|205|2|95|26|95|60|32|1|17|146|1|991|3...,14530|95|1156
6,0,13883743923141531481,20568201,29921,2,1,11198634,2,3,14530|95|1156,14530|95|1156|0|42|45|14530|95|1156,14530|95|1156|36|14530|4557|6|1|92|28|81|42|45...,14530|95|1156
7,0,4298118681424644510,7686578,385,1,1,16905873,2,4,121|3337|95|6,121|3337|95|6|0|4732|95|146|4079,8|81|123|205|2|95|26|95|60|32|1|17|146|1|991|3...,3337
8,0,4298118681424644510,7687047,385,1,1,5077836,2,3,1970|95|17,1970|95|17|0|4732|95|146|4079,8|81|123|205|2|95|26|95|60|32|1|17|146|1|991|3...,1970|4942
9,0,4298118681424644510,7686572,385,2,2,3519270,2,1,3337|95|126,3337|95|126|0|4732|95|146|4079,8|81|123|205|2|95|26|95|60|32|1|17|146|1|991|3...,304|20|3337|95


### Średni zbiór treningowy

In [174]:
D10M = pd.read_csv("D10M.tsv", sep = "\t")

In [175]:
D10M.head(20)

,Click,DisplayURL,AdId,AdvertiserId,Depth,Position,UserID,Gender,Age,AdKeyword_tokens,AdTitle_tokens,AdDescription_tokens,Query_tokens
0,0,4298118681424644510,7686695,385,3,3,490234,1,3,4133|95|17,4133|95|17|0|4732|95|146|4079,8|81|123|205|2|95|26|95|60|32|1|17|146|1|991|3...,4133
1,0,13677630321509009335,3517124,23778,3,1,490234,1,3,4133,145|65|3927|832|93,3683|4990|2793|11589|21|10741|26|16044|26|3168...,4133
2,0,11689327222955583742,21021375,27701,3,2,490234,1,3,4133,4133|95|1|339|125|21|83093,726|50|2218|1533|2275|4133|1299|509|95|2072|1|...,4133
3,0,4298118681424644510,7686695,385,1,1,16960371,2,2,4133|95|17,4133|95|17|0|4732|95|146|4079,8|81|123|205|2|95|26|95|60|32|1|17|146|1|991|3...,4133|4942
4,0,15132506310926074459,4424000,20940,1,1,3524325,1,3,121|4133|95,121|4133|95|8762|3957|4563|2233|192|28|138|3,62|1162|570|8|4133|95|1|81|102|1155|650|1255|1...,4133|4942
5,0,4298118681424644510,7686888,385,2,2,11198634,2,3,14530|882|167,14530|882|167|0|4732|95|146|4079,8|81|123|205|2|95|26|95|60|32|1|17|146|1|991|3...,14530|95|1156
6,0,13883743923141531481,20568201,29921,2,1,11198634,2,3,14530|95|1156,14530|95|1156|0|42|45|14530|95|1156,14530|95|1156|36|14530|4557|6|1|92|28|81|42|45...,14530|95|1156
7,0,4298118681424644510,7686578,385,1,1,16905873,2,4,121|3337|95|6,121|3337|95|6|0|4732|95|146|4079,8|81|123|205|2|95|26|95|60|32|1|17|146|1|991|3...,3337
8,0,4298118681424644510,7687047,385,1,1,5077836,2,3,1970|95|17,1970|95|17|0|4732|95|146|4079,8|81|123|205|2|95|26|95|60|32|1|17|146|1|991|3...,1970|4942
9,0,4298118681424644510,7686572,385,2,2,3519270,2,1,3337|95|126,3337|95|126|0|4732|95|146|4079,8|81|123|205|2|95|26|95|60|32|1|17|146|1|991|3...,304|20|3337|95


### Zbiór testowy

In [176]:
test = pd.read_csv("D5M_test_x.tsv", sep = "\t")

In [177]:
test.head(20)

,Click,DisplayURL,AdId,AdvertiserId,Depth,Position,UserID,Gender,Age,AdKeyword_tokens,AdTitle_tokens,AdDescription_tokens,Query_tokens
0,0,9860713142308876126,21197821,35826,2,1,9162163,2,1,143,162|5059|61|143|4|15|4185|462|49|3,143|1556|61|143|134|33|49|1|1157|1044|61|1|579...,143
1,0,9860713142308876126,21197821,35826,2,1,13329036,2,2,143,162|5059|61|143|4|15|4185|462|49|3,143|1556|61|143|134|33|49|1|1157|1044|61|1|579...,143
2,0,9860713142308876126,21197821,35826,2,1,295270,1,3,143,162|5059|61|143|4|15|4185|462|49|3,143|1556|61|143|134|33|49|1|1157|1044|61|1|579...,143
3,0,12892941488276077458,21460946,34043,2,2,6868758,2,5,143,143|1876|1275|6057|1030,143|1|6681|2|242|7277|2223|1|137|92|2312|1308|...,143
4,0,12892941488276077458,21460946,34043,2,2,2447751,1,1,143,143|1876|1275|6057|1030,143|1|6681|2|242|7277|2223|1|137|92|2312|1308|...,143
5,0,12892941488276077458,21460946,34043,2,1,13308359,2,5,143,143|1876|1275|6057|1030,143|1|6681|2|242|7277|2223|1|137|92|2312|1308|...,143
6,0,12892941488276077458,21460946,34043,2,2,13338529,1,6,143,143|1876|1275|6057|1030,143|1|6681|2|242|7277|2223|1|137|92|2312|1308|...,143
7,0,12892941488276077458,21460946,34043,2,1,6893672,1,2,143,143|1876|1275|6057|1030,143|1|6681|2|242|7277|2223|1|137|92|2312|1308|...,143
8,0,12892941488276077458,21460946,34043,3,3,9183599,1,1,143,143|1876|1275|6057|1030,143|1|6681|2|242|7277|2223|1|137|92|2312|1308|...,143
9,0,12892941488276077458,21460946,34043,2,1,6882275,2,1,143,143|1876|1275|6057|1030,6681|2|242|7277|2223|1|137|92|2312|1308|1353|8...,143


### Modelowanie

#### Chcemy przewidzieć, czy użytkownik kliknie na daną reklamę, czy nie. Wiemy, że w zbiorach D100k.tsv, D10M.tsv oraz D80M.tsv, było średnio 5% kliknięć użytkowników na reklamę.

#### Zmienną objaśnianą jest zmienna "Click".

#### Trenowałem modele na zbiorach D100k.tsv oraz D10M.tsv. Dla każdych modeli oraz zbiorów, dokonałem podziału na zbiór treningowy i walidacyjny w stosunku 80:20.

#### Dla modelu, który uznałem za najlepszy, tzn. były najlepsze wyniki AUC dla D100k i D10M, dokonałem predykcji na zbiorze testowym D5M_test_x.tsv

### Modele regresji liniowej

#### 1. Model z samą zmienną Age.

#### a) Mały zbiór

In [196]:
train_100k_1 = D100k.iloc[0:80000, :] 
test_100k_1 = D100k.iloc[80000:, :]    

In [197]:
model_maly_1 = smf.ols('Click ~ Age', data = train_100k_1)

In [198]:
model_maly_1 = model_maly_1.fit()

In [199]:
model_maly_1_summary = model_small_1.summary()
print(model_maly_1_summary)

                            OLS Regression Results                            
Dep. Variable:                  Click   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     9.370
Date:                Tue, 11 May 2021   Prob (F-statistic):            0.00221
Time:                        21:43:10   Log-Likelihood:                 8570.1
No. Observations:               80000   AIC:                        -1.714e+04
Df Residuals:                   79998   BIC:                        -1.712e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0445      0.002     23.613      0.0

In [200]:
type(model_maly_1)

statsmodels.regression.linear_model.RegressionResultsWrapper

In [201]:
model_maly_1.resid

0       -0.049487
1       -0.049487
2       -0.049487
3       -0.047816
4       -0.049487
           ...   
79995   -0.049487
79996   -0.049487
79997   -0.049487
79998   -0.049487
79999   -0.049487
Length: 80000, dtype: float64

In [202]:
pred_maly_1 = model_small_1.predict(test_100k_1.loc[:, ["Click", "Age"]])
pred_maly_1

80000    0.049487
80001    0.049487
80002    0.049487
80003    0.049487
80004    0.049487
           ...   
99995    0.049487
99996    0.049487
99997    0.052829
99998    0.047816
99999    0.051158
Length: 20000, dtype: float64

In [203]:
pred_maly_1.mean()
# 0.04987362976486753

0.04987362976486753

#### Czyli średnia predykcji wynosi prawie 0.05, czyli 5%, a więc się zgadza z średnią liczbą kliknięć użytkownika na reklamę.

In [204]:
sklearn.metrics.roc_auc_score(test_100k_1.loc[:,"Click"], pred_maly_1)
# 0.5202668887316323

0.5202668887316323

#### Zatem wartość AUC wynosi 0.520.

#### b) Średni zbiór 

In [187]:
train_10M_1 = D10M.iloc[0:80000, :] 
test_10M_1 = D10M.iloc[80000:, :]    

In [188]:
model_sredni_1 = smf.ols('Click ~ Age', data = train_10M_1)

In [190]:
model_sredni_1 = model_sredni_1.fit()

In [192]:
model_sredni_1.resid

0       -0.049487
1       -0.049487
2       -0.049487
3       -0.047816
4       -0.049487
           ...   
79995   -0.049487
79996   -0.049487
79997   -0.049487
79998   -0.049487
79999   -0.049487
Length: 80000, dtype: float64

In [193]:
pred_sredni_1 = model_sredni_1.predict(test_10M_1.loc[:, ["Click", "Age"]])
pred_sredni_1

80000      0.049487
80001      0.049487
80002      0.049487
80003      0.049487
80004      0.049487
             ...   
9999995    0.046145
9999996    0.046145
9999997    0.049487
9999998    0.052829
9999999    0.052829
Length: 9920000, dtype: float64

In [194]:
pred_sredni_1.mean()
# 0.050137931819387686

0.050137931819387686

In [195]:
sklearn.metrics.roc_auc_score(test_10M_1.loc[:,"Click"], pred_sredni_1)
# 0.5094500032414795

0.5094500032414795

#### AUC = 0.509

#### 2. Model ze zmienną "Age" i "Gender".

#### a) Średni zbiór 

In [205]:
model_maly_2 = smf.ols('Click ~ Age + Gender', data = train_100k_1)

In [206]:
model_maly_2 = model_maly_2.fit()

In [207]:
model_maly_2_summary = model_maly_2.summary()
print(model_maly_2_summary)

                            OLS Regression Results                            
Dep. Variable:                  Click   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     4.693
Date:                Tue, 11 May 2021   Prob (F-statistic):            0.00916
Time:                        21:43:58   Log-Likelihood:                 8570.1
No. Observations:               80000   AIC:                        -1.713e+04
Df Residuals:                   79997   BIC:                        -1.711e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0447      0.003     16.707      0.0

#### Widzimy, że obie zmienne są istotne.

In [208]:
model_maly_2.resid

0       -0.049566
1       -0.049566
2       -0.049566
3       -0.047700
4       -0.049566
           ...   
79995   -0.049566
79996   -0.049566
79997   -0.049566
79998   -0.049566
79999   -0.049566
Length: 80000, dtype: float64

In [209]:
pred_maly_2 = model_maly_2.predict(test_100k_1.loc[:, ["Click", "Age", "Gender"]])
pred_maly_2

80000    0.049566
80001    0.049566
80002    0.049566
80003    0.049566
80004    0.049566
           ...   
99995    0.049566
99996    0.049566
99997    0.052735
99998    0.047700
99999    0.051245
Length: 20000, dtype: float64

In [210]:
pred_maly_2.mean()
# 0.049866805068917774

0.049866805068917774

In [211]:
sklearn.metrics.roc_auc_score(test_100k_1.loc[:,"Click"], pred_maly_2)
# 0.5094500032414795

0.5199692471759081

#### AUC = 0.520

#### b) Średni zbiór 

In [213]:
model_sredni_2 = smf.ols('Click ~ Age + Gender', data = train_10M_1)
model_sredni_2 = model_sredni_2.fit()
pred_sredni_2 = model_sredni_2.predict(test_10M_1.loc[:, ["Click", "Age", "Gender"]])
pred_sredni_2.mean()
# 0.05014049764727051

0.05014049764727051

In [214]:
sklearn.metrics.roc_auc_score(test_10M_1.loc[:,"Click"], pred_sredni_2)
# 0.5084655293973371

0.5084655293973371

#### AUC = 0.508

#### 3. Model z samą zmienną "Gender"

#### a) Mały zbiór

In [219]:
model_maly_3 = smf.ols('Click ~ Gender', data = train_100k_1)
model_maly_3 = model_maly_3.fit()
pred_maly_3 = model_maly_3.predict(test_100k_1.loc[:, ["Click", "Gender"]])
pred_maly_3.mean()
# 0.04974794387199179

0.04974794387199179

In [220]:
sklearn.metrics.roc_auc_score(test_100k_1.loc[:,"Click"], pred_maly_3)
# 0.5046949656571331

0.5046949656571331

#### AUC = 0.505

#### b) Średni zbiór

In [222]:
model_sredni_3 = smf.ols('Click ~ Gender', data = train_10M_1)
model_sredni_3 = model_sredni_3.fit()
pred_sredni_3 = model_sredni_3.predict(test_10M_1.loc[:, ["Click", "Gender"]])
pred_sredni_3.mean()
# 0.04973632202593676

0.04973632202593676

In [223]:
sklearn.metrics.roc_auc_score(test_10M_1.loc[:,"Click"], pred_sredni_3)
# 0.5035258805338853

0.5035258805338853

#### AUC = 0.504

#### 4. Model z wszystkimi małymi zmiennymi: "Age", "Gender", "Depth", "Position"

#### a) Mały zbiór

In [398]:
model_maly_4 = smf.ols('Click ~ Age + Gender + Depth + Position', data = train_100k_1)
model_maly_4 = model_maly_4.fit()
pred_maly_4 = model_maly_4.predict(test_100k_1.loc[:, ["Click", "Age", "Gender", "Depth", "Position"]])
pred_maly_4.mean()
# 0.04951705040010446

0.04951705040010446

In [399]:
sklearn.metrics.roc_auc_score(test_100k_1.loc[:,"Click"], pred_maly_4)
# 0.6187734984617962

0.6187734984617962

#### AUC = 0.619

#### Przy uwzględnieniu wszystkich małych zmiennych widać więc wyraźną poprawę.

#### b) Średni zbiór

In [400]:
model_sredni_4 = smf.ols('Click ~ Age + Gender + Depth + Position', data = train_10M_1)
model_sredni_4 = model_sredni_4.fit()
pred_sredni_4 = model_sredni_4.predict(test_10M_1.loc[:, ["Click", "Age", "Gender", "Depth", "Position"]])
pred_sredni_4.mean()
# 0.05009517168821361

0.05009517168821361

In [401]:
sklearn.metrics.roc_auc_score(test_10M_1.loc[:,"Click"], pred_sredni_4)
# 0.5981106235060497

0.5981106235060497

#### AUC = 0.598

#### A więc w przypadku zbioru średniego, również widać wyraźną poprawę.

#### Generalnie, AUC na zbiorze walidacyjnym ze zbioru o 10 milionach wierszy wychodziło niższe niż AUC na zbiorze walidacyjnym ze zbioru o 100 tysiącach wierszy. Wynika to z tego, że w przypadku zbioru D10M, testujemy na dużo większym zbiorze.

### Modele z użyciem pakietu tensorflow

#### 5. Model z wszystkimi małymi zmiennymi: "Age", "Gender", Depth", "Position" - 1 warstwa z aktywacją za pomocą funkcji ReLU. Kompilowałem model z wykorzystaniem klasyfikatora stochastycznego spadku gradientu SGD z binarną funkcją straty krzyżowej entropii.

#### Liczba epok = 5

#### a) Mały zbiór

In [297]:
train_100k_5 = D100k.iloc[0:80000, :] 
test_100k_5 = D100k.iloc[80000:, :]

X_train_maly_5 = train_100k_5.loc[:, ["Depth", "Position", "Gender", "Age"]]
Y_train_maly_5 = train_100k_5.loc[:, "Click"]
                                        
X_test_maly_5 = test_100k_5.loc[:, ["Depth", "Position", "Gender", "Age"]]
Y_test_maly_5 = test_100k_5.loc[:, "Click"]                                        

In [298]:
model_maly_5 = tf.keras.models.Sequential()
model_maly_5.add(tf.keras.Input(shape = (4, ))) # Były 4 zmienne objaśniające, zatem dałem wymiar równy (4, ).
model_maly_5.add(tf.keras.layers.Dense(1, activation = "relu"))

In [299]:
model_maly_5.compile("sgd", loss = "binary_crossentropy")
model_maly_5.fit(X_train_maly_5, Y_train_maly_5, epochs = 5, validation_split = 0.2, verbose = 1)

Epoch 1/5
2000/2000 [==============================] - 2s 653us/step - loss: 0.9809 - val_loss: 0.8329
Epoch 2/5
2000/2000 [==============================] - 1s 639us/step - loss: 0.7447 - val_loss: 0.8329
Epoch 3/5
2000/2000 [==============================] - 1s 603us/step - loss: 0.7400 - val_loss: 0.8329
Epoch 4/5
2000/2000 [==============================] - 1s 596us/step - loss: 0.7405 - val_loss: 0.8329
Epoch 5/5
2000/2000 [==============================] - 1s 593us/step - loss: 0.7447 - val_loss: 0.8329


In [313]:
np.sum(model_maly_5.predict(X_test_maly_5) > 0.5)
# 0

0

In [301]:
pred_maly_5 = model_maly_5.predict(X_test_maly_5)

In [302]:
pred_maly_5

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [303]:
sklearn.metrics.roc_auc_score(Y_test_maly_5, pred_maly_5)
# 0.5

0.5

#### AUC = 0.500

#### b) Średni zbiór

In [304]:
train_10M_5 = D10M.iloc[0:80000, :] 
test_10M_5 = D10M.iloc[80000:, :]

X_train_sredni_5 = train_10M_5.loc[:, ["Depth", "Position", "Gender", "Age"]]
Y_train_sredni_5 = train_10M_5.loc[:, "Click"]
                                        
X_test_sredni_5 = test_10M_5.loc[:, ["Depth", "Position", "Gender", "Age"]]
Y_test_sredni_5 = test_10M_5.loc[:, "Click"]

In [305]:
model_sredni_5 = tf.keras.models.Sequential()
model_sredni_5.add(tf.keras.Input(shape = (4, ))) # Były 4 zmienne objaśniające, zatem dałem wymiar równy (4, ).
model_sredni_5.add(tf.keras.layers.Dense(1, activation = "relu"))

In [306]:
model_sredni_5.compile("sgd", loss = "binary_crossentropy")
model_sredni_5.fit(X_train_sredni_5, Y_train_sredni_5, epochs = 5, validation_split = 0.2, verbose = 1)

Epoch 1/5
2000/2000 [==============================] - 2s 641us/step - loss: 0.7342 - val_loss: 0.8329
Epoch 2/5
2000/2000 [==============================] - 1s 595us/step - loss: 0.7327 - val_loss: 0.8329
Epoch 3/5
2000/2000 [==============================] - 1s 595us/step - loss: 0.7577 - val_loss: 0.8329
Epoch 4/5
2000/2000 [==============================] - 1s 627us/step - loss: 0.7444 - val_loss: 0.8329
Epoch 5/5
2000/2000 [==============================] - 1s 628us/step - loss: 0.7380 - val_loss: 0.8329


In [307]:
np.sum(model_sredni_5.predict(X_test_sredni_5) > 0.5)
# 0

0

In [308]:
pred_sredni_5 = model_sredni_5.predict(X_test_sredni_5)

In [310]:
sklearn.metrics.roc_auc_score(Y_test_sredni_5, pred_sredni_5)
# 0.5

0.5

#### AUC = 0.500

#### 6. Model przy zwiększonej liczbie warstw z funkcją ReLU do 2 oraz dodatkowo z jedną warstwą przy aktywacji za pomocą funkcji sigmoidalnej.

#### a) Mały zbiór

In [336]:
model_maly_6 = tf.keras.models.Sequential()
model_maly_6.add(tf.keras.Input(shape = (4, ))) # Były 4 zmienne objaśniające, zatem dałem wymiar równy (4, ).
model_maly_6.add(tf.keras.layers.Dense(2, activation = "relu"))
model_maly_6.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
model_maly_6.compile("sgd", loss = "binary_crossentropy")
model_maly_6.fit(X_train_maly_5, Y_train_maly_5, epochs = 5, validation_split = 0.2, verbose = 1)

Epoch 1/5
2000/2000 [==============================] - 2s 691us/step - loss: 0.4210 - val_loss: 0.2102
Epoch 2/5
2000/2000 [==============================] - 1s 613us/step - loss: 0.1964 - val_loss: 0.2105
Epoch 3/5
2000/2000 [==============================] - 1s 623us/step - loss: 0.1924 - val_loss: 0.2100
Epoch 4/5
2000/2000 [==============================] - 1s 615us/step - loss: 0.1952 - val_loss: 0.2102
Epoch 5/5
2000/2000 [==============================] - 1s 606us/step - loss: 0.1925 - val_loss: 0.2097


In [337]:
np.sum(model_maly_6.predict(X_test_maly_5) > 0.5)
# 0

0

In [338]:
pred_maly_6 = model_maly_6.predict(X_test_maly_5)
sklearn.metrics.roc_auc_score(Y_test_maly_5, pred_maly_6)
# 0.5761708885119532

0.5761708885119532

#### AUC = 0.576

#### Widać więc znaczącą poprawę w AUC.

#### b) Średni zbiór

In [339]:
model_sredni_6 = tf.keras.models.Sequential()
model_sredni_6.add(tf.keras.Input(shape = (4, ))) # Były 4 zmienne objaśniające, zatem dałem wymiar równy (4, ).
model_sredni_6.add(tf.keras.layers.Dense(2, activation = "relu"))
model_sredni_6.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
model_sredni_6.compile("sgd", loss = "binary_crossentropy")
model_sredni_6.fit(X_train_sredni_5, Y_train_sredni_5, epochs = 5, validation_split = 0.2, verbose = 1)

Epoch 1/5
2000/2000 [==============================] - 2s 664us/step - loss: 0.2121 - val_loss: 0.2176
Epoch 2/5
2000/2000 [==============================] - 1s 607us/step - loss: 0.1957 - val_loss: 0.2168
Epoch 3/5
2000/2000 [==============================] - 1s 615us/step - loss: 0.1991 - val_loss: 0.2164
Epoch 4/5
2000/2000 [==============================] - 1s 624us/step - loss: 0.1976 - val_loss: 0.2153
Epoch 5/5
2000/2000 [==============================] - 1s 588us/step - loss: 0.1962 - val_loss: 0.2142


In [340]:
np.sum(model_sredni_6.predict(X_test_sredni_5) > 0.5)
# 0

0

In [341]:
pred_sredni_6 = model_sredni_6.predict(X_test_sredni_5)
sklearn.metrics.roc_auc_score(Y_test_sredni_5, pred_sredni_6)
# 0.5616961310938284

0.5616961310938284

#### AUC = 0.562

#### Stąd w przypadku zbioru średniego, również widać wyraźną poprawę.

#### 7. Model przy zwiększonej liczbie warstw z funkcją ReLU z 2 do 3.

#### a) Mały zbiór

In [342]:
model_maly_7 = tf.keras.models.Sequential()
model_maly_7.add(tf.keras.Input(shape = (4, ))) # Były 4 zmienne objaśniające, zatem dałem wymiar równy (4, ).
model_maly_7.add(tf.keras.layers.Dense(3, activation = "relu"))
model_maly_7.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
model_maly_7.compile("sgd", loss = "binary_crossentropy")
model_maly_7.fit(X_train_maly_5, Y_train_maly_5, epochs = 5, validation_split = 0.2, verbose = 1)

Epoch 1/5
2000/2000 [==============================] - 2s 667us/step - loss: 0.3023 - val_loss: 0.2174
Epoch 2/5
2000/2000 [==============================] - 1s 629us/step - loss: 0.1928 - val_loss: 0.2126
Epoch 3/5
2000/2000 [==============================] - 1s 635us/step - loss: 0.1936 - val_loss: 0.2117
Epoch 4/5
2000/2000 [==============================] - 1s 624us/step - loss: 0.1920 - val_loss: 0.2110
Epoch 5/5
2000/2000 [==============================] - 1s 631us/step - loss: 0.1914 - val_loss: 0.2107


In [343]:
np.sum(model_maly_7.predict(X_test_maly_5) > 0.5)
# 0

0

In [344]:
pred_maly_7 = model_maly_7.predict(X_test_maly_5)
sklearn.metrics.roc_auc_score(Y_test_maly_5, pred_maly_7)
# 0.5889738082183392

0.5889738082183392

#### AUC = 0.589

#### Zatem AUC jeszcze trochę wzrosło.

#### b) Średni zbiór

In [345]:
model_sredni_7 = tf.keras.models.Sequential()
model_sredni_7.add(tf.keras.Input(shape = (4, ))) # Były 4 zmienne objaśniające, zatem dałem wymiar równy (4, ).
model_sredni_7.add(tf.keras.layers.Dense(3, activation = "relu"))
model_sredni_7.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
model_sredni_7.compile("sgd", loss = "binary_crossentropy")
model_sredni_7.fit(X_train_sredni_5, Y_train_sredni_5, epochs = 5, validation_split = 0.2, verbose = 1)

Epoch 1/5
2000/2000 [==============================] - 2s 665us/step - loss: 0.2942 - val_loss: 0.2252
Epoch 2/5
2000/2000 [==============================] - 1s 631us/step - loss: 0.2009 - val_loss: 0.2194
Epoch 3/5
2000/2000 [==============================] - 1s 649us/step - loss: 0.1961 - val_loss: 0.2161
Epoch 4/5
2000/2000 [==============================] - 1s 672us/step - loss: 0.1935 - val_loss: 0.2142
Epoch 5/5
2000/2000 [==============================] - 1s 620us/step - loss: 0.1956 - val_loss: 0.2132


In [346]:
np.sum(model_sredni_7.predict(X_test_sredni_5) > 0.5)
# 0

0

In [348]:
pred_sredni_7 = model_sredni_7.predict(X_test_sredni_5)
sklearn.metrics.roc_auc_score(Y_test_sredni_5, pred_sredni_7)
# 0.5644865906420553

0.5644865906420553

#### AUC = 0.564

#### Zatem dla zbioru średniego, również AUC trochę się zwiększyło.

#### 8. Model przy zwiększonej liczbie epok z 5 do 10

#### a) Mały zbiór

In [349]:
model_maly_8 = tf.keras.models.Sequential()
model_maly_8.add(tf.keras.Input(shape = (4, ))) # Były 4 zmienne objaśniające, zatem dałem wymiar równy (4, ).
model_maly_8.add(tf.keras.layers.Dense(3, activation = "relu"))
model_maly_8.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
model_maly_8.compile("sgd", loss = "binary_crossentropy")
model_maly_8.fit(X_train_maly_5, Y_train_maly_5, epochs = 10, validation_split = 0.2, verbose = 1)

Epoch 1/10
2000/2000 [==============================] - 2s 666us/step - loss: 0.2245 - val_loss: 0.2201
Epoch 2/10
2000/2000 [==============================] - 1s 601us/step - loss: 0.2037 - val_loss: 0.2156
Epoch 3/10
2000/2000 [==============================] - 1s 604us/step - loss: 0.1958 - val_loss: 0.2130
Epoch 4/10
2000/2000 [==============================] - 1s 600us/step - loss: 0.1958 - val_loss: 0.2126
Epoch 5/10
2000/2000 [==============================] - 1s 607us/step - loss: 0.1935 - val_loss: 0.2109
Epoch 6/10
2000/2000 [==============================] - 1s 608us/step - loss: 0.1911 - val_loss: 0.2108
Epoch 7/10
2000/2000 [==============================] - 1s 604us/step - loss: 0.1963 - val_loss: 0.2102
Epoch 8/10
2000/2000 [==============================] - 1s 608us/step - loss: 0.1932 - val_loss: 0.2097
Epoch 9/10
2000/2000 [==============================] - 1s 612us/step - loss: 0.1912 - val_loss: 0.2094
Epoch 10/10
2000/2000 [==============================] - 1s 627u

In [350]:
np.sum(model_maly_8.predict(X_test_maly_5) > 0.5)
# 0

0

In [351]:
pred_maly_8 = model_maly_8.predict(X_test_maly_5)
sklearn.metrics.roc_auc_score(Y_test_maly_5, pred_maly_8)
# 0.5998540011758232

0.5998540011758232

#### AUC = 0.600

#### Stąd zwiększenie liczby epok poprawiło rezultaty.

#### b) Średni zbiór

In [356]:
model_sredni_8 = tf.keras.models.Sequential()
model_sredni_8.add(tf.keras.Input(shape = (4, ))) # Były 4 zmienne objaśniające, zatem dałem wymiar równy (4, ).
model_sredni_8.add(tf.keras.layers.Dense(3, activation = "relu"))
model_sredni_8.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
model_sredni_8.compile("sgd", loss = "binary_crossentropy")
model_sredni_8.fit(X_train_sredni_5, Y_train_sredni_5, epochs = 10, validation_split = 0.2, verbose = 1)

Epoch 1/10
2000/2000 [==============================] - 2s 661us/step - loss: 0.2182 - val_loss: 0.2211
Epoch 2/10
2000/2000 [==============================] - 1s 612us/step - loss: 0.1989 - val_loss: 0.2177
Epoch 3/10
2000/2000 [==============================] - 1s 610us/step - loss: 0.1996 - val_loss: 0.2156
Epoch 4/10
2000/2000 [==============================] - 1s 604us/step - loss: 0.1976 - val_loss: 0.2144
Epoch 5/10
2000/2000 [==============================] - 1s 605us/step - loss: 0.1946 - val_loss: 0.2143
Epoch 6/10
2000/2000 [==============================] - 1s 607us/step - loss: 0.1937 - val_loss: 0.2137
Epoch 7/10
2000/2000 [==============================] - 1s 608us/step - loss: 0.1950 - val_loss: 0.2124
Epoch 8/10
2000/2000 [==============================] - 1s 613us/step - loss: 0.1968 - val_loss: 0.2119
Epoch 9/10
2000/2000 [==============================] - 1s 611us/step - loss: 0.1913 - val_loss: 0.2110
Epoch 10/10
2000/2000 [==============================] - 1s 598u

In [357]:
np.sum(model_sredni_8.predict(X_test_sredni_5) > 0.5)
# 0

0

In [358]:
pred_sredni_8 = model_sredni_8.predict(X_test_sredni_5)
sklearn.metrics.roc_auc_score(Y_test_sredni_5, pred_sredni_8)
# 0.567679172442108

0.567679172442108

#### AUC = 0.568

#### Zatem dla zbioru średniego także AUC trochę się poprawiło.

#### 9. Model ze zwiększoną liczbą epok z 10 do 15

#### a) Mały zbiór

In [362]:
model_maly_9 = tf.keras.models.Sequential()
model_maly_9.add(tf.keras.Input(shape = (4, ))) # Były 4 zmienne objaśniające, zatem dałem wymiar równy (4, ).
model_maly_9.add(tf.keras.layers.Dense(3, activation = "relu"))
model_maly_9.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
model_maly_9.compile("sgd", loss = "binary_crossentropy")
model_maly_9.fit(X_train_maly_5, Y_train_maly_5, epochs = 15, validation_split = 0.2, verbose = 1)

Epoch 1/15
2000/2000 [==============================] - 2s 658us/step - loss: 0.2268 - val_loss: 0.2184
Epoch 2/15
2000/2000 [==============================] - 1s 614us/step - loss: 0.1964 - val_loss: 0.2154
Epoch 3/15
2000/2000 [==============================] - 1s 616us/step - loss: 0.1925 - val_loss: 0.2147
Epoch 4/15
2000/2000 [==============================] - 1s 601us/step - loss: 0.1966 - val_loss: 0.2134
Epoch 5/15
2000/2000 [==============================] - 1s 608us/step - loss: 0.1942 - val_loss: 0.2123
Epoch 6/15
2000/2000 [==============================] - 1s 610us/step - loss: 0.1922 - val_loss: 0.2124
Epoch 7/15
2000/2000 [==============================] - 1s 603us/step - loss: 0.1930 - val_loss: 0.2117
Epoch 8/15
2000/2000 [==============================] - 1s 612us/step - loss: 0.1947 - val_loss: 0.2107
Epoch 9/15
2000/2000 [==============================] - 1s 599us/step - loss: 0.1942 - val_loss: 0.2113
Epoch 10/15
2000/2000 [==============================] - 1s 613u

In [363]:
np.sum(model_maly_9.predict(X_test_maly_5) > 0.5)
# 0

0

In [364]:
pred_maly_9 = model_maly_9.predict(X_test_maly_5)
sklearn.metrics.roc_auc_score(Y_test_maly_5, pred_maly_9)
# 0.6121626147575743

0.6121626147575743

#### AUC = 0.612

#### b) Średni zbiór

In [365]:
model_sredni_9 = tf.keras.models.Sequential()
model_sredni_9.add(tf.keras.Input(shape = (4, ))) # Były 4 zmienne objaśniające, zatem dałem wymiar równy (4, ).
model_sredni_9.add(tf.keras.layers.Dense(3, activation = "relu"))
model_sredni_9.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
model_sredni_9.compile("sgd", loss = "binary_crossentropy")
model_sredni_9.fit(X_train_sredni_5, Y_train_sredni_5, epochs = 15, validation_split = 0.2, verbose = 1)

Epoch 1/15
2000/2000 [==============================] - 2s 691us/step - loss: 0.2141 - val_loss: 0.2229
Epoch 2/15
2000/2000 [==============================] - 1s 640us/step - loss: 0.1995 - val_loss: 0.2173
Epoch 3/15
2000/2000 [==============================] - 1s 604us/step - loss: 0.1938 - val_loss: 0.2140
Epoch 4/15
2000/2000 [==============================] - 1s 615us/step - loss: 0.1938 - val_loss: 0.2128
Epoch 5/15
2000/2000 [==============================] - 1s 600us/step - loss: 0.1917 - val_loss: 0.2119
Epoch 6/15
2000/2000 [==============================] - 1s 604us/step - loss: 0.1940 - val_loss: 0.2115
Epoch 7/15
2000/2000 [==============================] - 1s 606us/step - loss: 0.1980 - val_loss: 0.2113
Epoch 8/15
2000/2000 [==============================] - 1s 603us/step - loss: 0.1938 - val_loss: 0.2111
Epoch 9/15
2000/2000 [==============================] - 1s 611us/step - loss: 0.1923 - val_loss: 0.2105
Epoch 10/15
2000/2000 [==============================] - 1s 638u

In [366]:
np.sum(model_sredni_9.predict(X_test_sredni_5) > 0.5)
# 0

0

In [367]:
pred_sredni_9 = model_sredni_9.predict(X_test_sredni_5)
sklearn.metrics.roc_auc_score(Y_test_sredni_5, pred_sredni_9)
# 0.5845638568854904

0.5845638568854904

#### AUC = 0.585

#### Stąd, w obu zbiorach, AUC wzrosło.

#### 10. Model ze zwiększoną liczbą warstw z funkcją ReLU z 3 do 4

#### a) Mały zbiór

In [368]:
model_maly_10 = tf.keras.models.Sequential()
model_maly_10.add(tf.keras.Input(shape = (4, ))) # Były 4 zmienne objaśniające, zatem dałem wymiar równy (4, ).
model_maly_10.add(tf.keras.layers.Dense(4, activation = "relu"))
model_maly_10.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
model_maly_10.compile("sgd", loss = "binary_crossentropy")
model_maly_10.fit(X_train_maly_5, Y_train_maly_5, epochs = 15, validation_split = 0.2, verbose = 1)

Epoch 1/15
2000/2000 [==============================] - 2s 657us/step - loss: 0.2399 - val_loss: 0.2213
Epoch 2/15
2000/2000 [==============================] - 1s 601us/step - loss: 0.1990 - val_loss: 0.2142
Epoch 3/15
2000/2000 [==============================] - 1s 629us/step - loss: 0.1975 - val_loss: 0.2131
Epoch 4/15
2000/2000 [==============================] - 1s 611us/step - loss: 0.1881 - val_loss: 0.2122
Epoch 5/15
2000/2000 [==============================] - 1s 608us/step - loss: 0.1925 - val_loss: 0.2120
Epoch 6/15
2000/2000 [==============================] - 1s 626us/step - loss: 0.1905 - val_loss: 0.2114
Epoch 7/15
2000/2000 [==============================] - 1s 614us/step - loss: 0.1910 - val_loss: 0.2112
Epoch 8/15
2000/2000 [==============================] - 1s 600us/step - loss: 0.1940 - val_loss: 0.2105
Epoch 9/15
2000/2000 [==============================] - 1s 603us/step - loss: 0.1911 - val_loss: 0.2111
Epoch 10/15
2000/2000 [==============================] - 1s 610u

In [370]:
np.sum(model_maly_10.predict(X_test_maly_5) > 0.5)
# 0

0

In [371]:
pred_maly_10 = model_maly_10.predict(X_test_maly_5)
sklearn.metrics.roc_auc_score(Y_test_maly_5, pred_maly_10)
# 0.6011917422713919

0.6011917422713919

#### AUC = 0.601

#### b) Średni zbiór

In [372]:
model_sredni_10 = tf.keras.models.Sequential()
model_sredni_10.add(tf.keras.Input(shape = (4, ))) # Były 4 zmienne objaśniające, zatem dałem wymiar równy (4, ).
model_sredni_10.add(tf.keras.layers.Dense(4, activation = "relu"))
model_sredni_10.add(tf.keras.layers.Dense(1, activation = "sigmoid"))
model_sredni_10.compile("sgd", loss = "binary_crossentropy")
model_sredni_10.fit(X_train_sredni_5, Y_train_sredni_5, epochs = 15, validation_split = 0.2, verbose = 1)

Epoch 1/15
2000/2000 [==============================] - 2s 683us/step - loss: 0.2012 - val_loss: 0.2175
Epoch 2/15
2000/2000 [==============================] - 1s 604us/step - loss: 0.1947 - val_loss: 0.2171
Epoch 3/15
2000/2000 [==============================] - 1s 600us/step - loss: 0.1925 - val_loss: 0.2147
Epoch 4/15
2000/2000 [==============================] - 1s 632us/step - loss: 0.1954 - val_loss: 0.2135
Epoch 5/15
2000/2000 [==============================] - 1s 593us/step - loss: 0.1922 - val_loss: 0.2127
Epoch 6/15
2000/2000 [==============================] - 1s 617us/step - loss: 0.1955 - val_loss: 0.2123
Epoch 7/15
2000/2000 [==============================] - 1s 607us/step - loss: 0.1935 - val_loss: 0.2115
Epoch 8/15
2000/2000 [==============================] - 1s 604us/step - loss: 0.1918 - val_loss: 0.2114
Epoch 9/15
2000/2000 [==============================] - 1s 612us/step - loss: 0.1929 - val_loss: 0.2107
Epoch 10/15
2000/2000 [==============================] - 1s 598u

In [373]:
np.sum(model_sredni_10.predict(X_test_sredni_5) > 0.5)
# 0

0

In [374]:
pred_sredni_10 = model_sredni_10.predict(X_test_sredni_5)
sklearn.metrics.roc_auc_score(Y_test_sredni_5, pred_sredni_10)
# 0.586129285686636

0.586129285686636

#### AUC = 0.586

#### Zatem, w przypadku małego zbioru, AUC, trochę spadło, a w przypadku średniego zbioru, bardzo niewiele wzrosło. Stąd nie warto zwiększać liczby warstw przy funkcji ReLU z 3 do 4. Optymalna liczba warstw to 3.
#### Natomiast w przypadku liczby epok, wybrałem liczbę 15.

### Później spróbowałem dodać zmienną "AdvertiserId" do modelu z wykorzystaniem warstwy Embedding.

#### a) Mały zbiór

In [379]:
train_maly_11 = D100k.iloc[0:80000, :] 
test_maly_11 = D100k.iloc[80000:, :]

X_train_maly_11 = train_maly_11.loc[:, ["Depth", "Position", "Gender", "Age", "AdvertiserId"]]
Y_train_maly_11 = train_maly_11.loc[:, "Click"]
                                        
X_test_maly_11 = test_maly_11.loc[:, ["Depth", "Position", "Gender", "Age", "AdvertiserId"]]
Y_test_maly_11 = test_maly_11.loc[:, "Click"]  

In [382]:
print("Największa liczba odkreślająca identyfikator:")
print(max(x for x in D100k.AdvertiserId))

Największa liczba odkreślająca identyfikator:
39083


In [385]:
vocab_size = 39083 + 1

# Gdybym nie dodał 1 do największej liczby określającej identyfikator, to miałbym następujący komunikat:
# InvalidArgumentError:  indices[67,4] = 39083 is not in [0, 39083)
# 	 [[node sequential_43/embedding_29/embedding_lookup (defined at <ipython-input-383-ac7e3b8f2c50>:13) ]] [Op:__inference_train_function_5399745]

# Errors may have originated from an input operation.
# Input Source operations connected to node sequential_43/embedding_29/embedding_lookup:
#  sequential_43/embedding_29/embedding_lookup/5399179 (defined at C:\Users\Kuboz\anaconda3\lib\contextlib.py:113)

# Function call stack:
# train_function

model_maly_11 = tf.keras.models.Sequential()


model_maly_11.add(Embedding(vocab_size,1))

model_maly_11.add(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=False))
model_maly_11.add(tf.keras.layers.Dense(3, activation = "relu"))
model_maly_11.add(Dense(1, activation='sigmoid'))

model_maly_11.compile("sgd", loss = "binary_crossentropy")
model_maly_11.fit(X_train_maly_11, Y_train_maly_11,
              batch_size=128,
              epochs=15,
              verbose=1,
              validation_data=(X_test_maly_11, Y_test_maly_11))

Epoch 1/15
625/625 [==============================] - 2s 2ms/step - loss: 0.3241 - val_loss: 0.1889
Epoch 2/15
625/625 [==============================] - 1s 2ms/step - loss: 0.2031 - val_loss: 0.1887
Epoch 3/15
625/625 [==============================] - 1s 2ms/step - loss: 0.1994 - val_loss: 0.1884
Epoch 4/15
625/625 [==============================] - 1s 2ms/step - loss: 0.2008 - val_loss: 0.1886
Epoch 5/15
625/625 [==============================] - 1s 2ms/step - loss: 0.1990 - val_loss: 0.1885
Epoch 6/15
625/625 [==============================] - 1s 2ms/step - loss: 0.1984 - val_loss: 0.1886
Epoch 7/15
625/625 [==============================] - 1s 2ms/step - loss: 0.1976 - val_loss: 0.1884
Epoch 8/15
625/625 [==============================] - 1s 2ms/step - loss: 0.1981 - val_loss: 0.1883
Epoch 9/15
625/625 [==============================] - 1s 2ms/step - loss: 0.1999 - val_loss: 0.1885
Epoch 10/15
625/625 [==============================] - 1s 2ms/step - loss: 0.1989 - val_loss: 0.1883

In [386]:
pred_maly_11 = model_maly_11.predict(X_test_maly_11)

In [387]:
pred_maly_11

array([[0.0430811 ],
       [0.03937787],
       [0.04359093],
       ...,
       [0.04219833],
       [0.04068637],
       [0.04407543]], dtype=float32)

In [388]:
pred_maly_11.shape
# (20000, 1)

(20000, 1)

#### Mamy więc macierz wymiaru 20000 x 1. Wyciągamy wszystkie elementy z kolumny.

In [389]:
pred_maly_11 = pred_maly_11[:, 0]

In [390]:
sklearn.metrics.roc_auc_score(Y_test_maly_11, pred_maly_11)
# 0.5682481198979753

0.5682481198979753

#### AUC = 0.568

#### b) Średni zbiór

In [393]:
train_sredni_11 = D10M.iloc[0:80000, :] 
test_sredni_11 = D10M.iloc[80000:, :]

X_train_sredni_11 = train_sredni_11.loc[:, ["Depth", "Position", "Gender", "Age", "AdvertiserId"]]
Y_train_sredni_11 = train_sredni_11.loc[:, "Click"]
                                        
X_test_sredni_11 = test_sredni_11.loc[:, ["Depth", "Position", "Gender", "Age", "AdvertiserId"]]
Y_test_sredni_11 = test_sredni_11.loc[:, "Click"]  

In [394]:
print("Największa liczba odkreślająca identyfikator:")
print(max(x for x in D10M.AdvertiserId))

Największa liczba odkreślająca identyfikator:
39164


In [395]:
vocab_size = 39164 + 1

model_sredni_11 = tf.keras.models.Sequential()

model_sredni_11.add(Embedding(vocab_size,1))

model_sredni_11.add(SimpleRNN(64, dropout=0.2, recurrent_dropout=0.2, return_sequences=False))
model_sredni_11.add(tf.keras.layers.Dense(3, activation = "relu"))
model_sredni_11.add(Dense(1, activation='sigmoid'))

model_sredni_11.compile("sgd", loss = "binary_crossentropy")
model_sredni_11.fit(X_train_sredni_11, Y_train_sredni_11,
              batch_size=128,
              epochs=15,
              verbose=1,
              validation_data=(X_test_sredni_11, Y_test_sredni_11))

Epoch 1/15
625/625 [==============================] - 56s 89ms/step - loss: 0.2978 - val_loss: 0.1891
Epoch 2/15
625/625 [==============================] - 56s 90ms/step - loss: 0.1975 - val_loss: 0.1890
Epoch 3/15
625/625 [==============================] - 56s 89ms/step - loss: 0.1981 - val_loss: 0.1891
Epoch 4/15
625/625 [==============================] - 54s 87ms/step - loss: 0.1995 - val_loss: 0.1890
Epoch 5/15
625/625 [==============================] - 55s 88ms/step - loss: 0.1979 - val_loss: 0.1890
Epoch 6/15
625/625 [==============================] - 53s 85ms/step - loss: 0.2000 - val_loss: 0.1890
Epoch 7/15
625/625 [==============================] - 53s 85ms/step - loss: 0.2017 - val_loss: 0.1892
Epoch 8/15
625/625 [==============================] - 53s 84ms/step - loss: 0.1986 - val_loss: 0.1890
Epoch 9/15
625/625 [==============================] - 53s 86ms/step - loss: 0.1969 - val_loss: 0.1889
Epoch 10/15
625/625 [==============================] - 54s 86ms/step - loss: 0.199

In [396]:
pred_sredni_11 = model_sredni_11.predict(X_test_sredni_11)
pred_sredni_11 = pred_sredni_11[:, 0]
sklearn.metrics.roc_auc_score(Y_test_sredni_11, pred_sredni_11)
# 0.560587675159211

0.560587675159211

#### AUC = 0.561

#### Stąd dodanie tej zmiennej indykatorowej, nie poprawiło, a nawet trochę pogorszyło AUC. W dodatku, dla zbioru D10M, dopasowywanie modelu trwało bardzo długo (ponad 1 minutę zajęło wykonanie pojedynczej epoki), więc w sumie z 15 epokami, całe dopasowanie zajęło 15 minut.

#### Zatem ostatecznie wybieram model 10, tzn. model z wszystkimi małymi zmiennymi: "Age", "Gender", Depth", "Position" trenowany na zbiorze D10M, z 3 warstwami z aktywacją za pomocą funkcji ReLU i 1 warstwą z funkcją sigmoidalną, gdzie kompilowałem model z wykorzystaniem klasyfikatora stochastycznego spadku gradientu SGD z binarną funkcją straty krzyżowej entropii, z liczbą epok równą 15.

#### Można też zauważyć, że dodanie zmiennych Depth i Position, znacząco poprawiły wartość AUC.

#### Predykcje na zbiorze testowym D5M_test_x.tsv

In [403]:
X_test= test.loc[:, ["Depth", "Position", "Gender", "Age"]]
Y_test = test.loc[:, "Click"]       

In [404]:
pred_test = model_sredni_10.predict(X_test)

In [405]:
pred_test[:20]

array([[0.05961379],
       [0.05903229],
       [0.06689313],
       [0.02549398],
       [0.03053153],
       [0.05731967],
       [0.02902651],
       [0.06754649],
       [0.01508611],
       [0.05961379],
       [0.03022456],
       [0.05903229],
       [0.06754649],
       [0.0350883 ],
       [0.02992064],
       [0.02992064],
       [0.02992064],
       [0.02961966],
       [0.02628222],
       [0.06689313]], dtype=float32)

In [414]:
pred_test.shape

(5000000,)

In [413]:
pred_test = pred_test[:, 0]

In [428]:
pred_test

array([0.05961379, 0.05903229, 0.06689313, ..., 0.06624568, 0.02961966,
       0.02960917], dtype=float32)

In [417]:
pred_test.mean()
# 0.047198534

0.047198534

#### A więc wiemy, że średnio prawdopodobieństwo wynosi 0.047, czyli jest w miarę dobrze zbliżone do prawdopodobieństwa 0.05.

#### Zapisuję swoje otrzymane wyniki do pliku .txt.

In [431]:
plik = open("Predykcje_JZ.txt", "w")
plik.write("Jakub_Zbrzezny" + "\n")
for i in pred_test:
    plik.write(str(i) + "\n")